In [1]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow-io

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [ ]:
print(tf.__version__)
print(tfio.__version__)

In [9]:
# Define the configuration
class Config:
    SR = 16000  # Sampling rate expected by YAMNet
    ROOT_FOLDER = r"C:\Users\KimDongyoung\Downloads\SW중심대학"  # Update this to your dataset path
    N_CLASSES = 2
    BATCH_SIZE = 16  # Reduce batch size to avoid memory issues
    N_EPOCHS = 10
    LR = 1e-4
    SEED = 42

CONFIG = Config()

def seed_everything(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(CONFIG.SEED)

In [10]:
# Load CSV files
train_df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'train.csv'))
test_df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'test.csv'))
sample_submission_df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'sample_submission.csv'))

# Update paths in DataFrames
train_df['path'] = train_df['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x[2:]))
test_df['path'] = test_df['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x[2:]))

# Split the training data
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=CONFIG.SEED)

In [12]:
def preprocess_audio(file_path, target_sample_rate=16000, target_length=16000*5):  # Assume 5 seconds as the target length
    audio = tfio.audio.AudioIOTensor(file_path)
    audio_tensor = tf.cast(audio.to_tensor(), tf.float32) / 32768.0  # Normalize to [-1.0, 1.0]
    audio_resampled = tfio.audio.resample(audio_tensor, rate_in=audio.rate, rate_out=target_sample_rate)
    audio_resized = tf.image.resize_with_pad(audio_resampled, target_length, 1)
    return audio_resized.numpy().flatten()

def get_features(df, train_mode=True, target_length=16000*5):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        file_path = row['path']
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        audio = preprocess_audio(file_path, target_sample_rate=CONFIG.SR, target_length=target_length)
        features.append(audio)
        if train_mode:
            label = row['label']
            label_vector = 0 if label == 'fake' else 1
            labels.append(label_vector)
    if train_mode:
        return np.array(features), np.array(labels)
    return np.array(features)

In [13]:
train_features, train_labels = get_features(train_data, True)
val_features, val_labels = get_features(val_data, True)

  0%|                                                                                        | 0/44350 [00:01<?, ?it/s]


NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['C:\\Users\\KimDongyoung\\anaconda\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so']
caused by: ['C:\\Users\\KimDongyoung\\anaconda\\Lib\\site-packages\\tensorflow_io\\python\\ops\\libtensorflow_io.so not found']